# 10-Transforming Data

In this notebook we do a few things with the [Adult Income Dataset](https://www.kaggle.com/uciml/adult-census-income) to illustrate working with relationships between variables.



In [ ]:
# Includes and Standard Magic...
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd
# Load Stats
from scipy import stats
import seaborn as sns

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.style.use('fivethirtyeight')

# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Show a ludicrus number of rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Supress scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# Load the data
# Data from here: https://github.com/joshrosson/NBASalaryPredictions
df_nba = pd.read_csv("./data/nba_stats.csv")
df_nba.head()

Is the data for NBA Salary skewed?  Why?

In [ ]:
# Is the salary skewed?
df_nba['Salary'].plot.hist()

In [ ]:
df_nba['Salary'].describe()

In [ ]:
# Transformation functions..
x = np.linspace(0.0001, 200, 500)
plt.plot(x**1.5, label="$x^{1.5}$")
plt.plot(x**1, label="f(x)")
plt.plot(x**0.2, label="$x^{0.2}$")
#plt.plot(np.log(x), label="log(x)")
plt.legend(loc='best')

In [ ]:
# Apply a few functions...
(df_nba['Salary']**2).plot.hist()
plt.show()
(df_nba['Salary']**0.2).plot.hist()
plt.show()
(np.log(df_nba['Salary'])).plot.hist()
plt.show()

In [ ]:
# Uskewing the data can help us to see relationships.
# Try 2, 1, 0.2

plt.scatter((df_nba['Salary']**1.0), df_nba['PTS'])

In [ ]:
# Z-standardize and replot.
df_nba['std_salary'] = (df_nba['Salary'] - df_nba['Salary'].mean()) / df_nba['Salary'].std()
df_nba['std_pts'] = (df_nba['PTS'] - df_nba['PTS'].mean()) / df_nba['PTS'].std()


In [ ]:
# Plot Salary v. Assists..
# Can see a bit more distribution and units are interpretable!
df_nba.plot.scatter(x='std_salary', y='std_pts')

In [ ]:
# Compute the whole matrix..
df_nba.corr()

In [ ]:
# Compute Some Cross Correlations...
plt.figure(figsize = (16,5))
sns.heatmap(df_nba.corr(), vmin=-1, vmax=1, cmap=sns.diverging_palette(20, 220, as_cmap=True))

# Find the closest players!


In [ ]:
# Get a smaller set, drop NA's and get dummies...
df_comp = df_nba[['Name', 'Pos', 'Salary', 'PTS', 'TRB']].copy()
df_comp.dropna(inplace=True)
df_comp.reset_index(drop=True, inplace=True)
df_comp

In [ ]:
# get dummies
df_ml = pd.get_dummies(df_comp[['Pos', 'Salary', 'PTS', 'TRB']])
df_ml

We're going to start using [SKLearn](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics) we'll get more into it as we go!

In [ ]:
# Use SKLEARN to do soe distances.
from sklearn.metrics import pairwise_distances
D = pairwise_distances(df_ml, metric="euclidean")
D.shape

In [ ]:
# Find someone intersting...
df_comp[(df_comp['Name'] == 'Anthony Davis')]

In [ ]:
D[8416, :]

In [ ]:
# Whose the closest
D[8416, :].argmin()

In [ ]:
# Wait... that's me...
np.fill_diagonal(D, np.inf)

In [ ]:
D[8416, :].argmin()

In [ ]:
df_comp.loc[8547]

## Probability and Code!

Note we're using [Numpy's probability functions](https://numpy.org/doc/stable/reference/random/index.html), you could also use [Python's](https://docs.python.org/3/library/random.html)

In [ ]:
# Let's make a probability distribution:
outcomes = list(range(1,7))
outcomes

In [ ]:
#Simulate an outcome..
np.random.choice(outcomes)

In [ ]:
# Do it a lot...
np.random.choice(outcomes, 20)

In [ ]:
# Graph it!
results = pd.DataFrame(np.random.choice(outcomes, 1000))
results.plot.hist(bins=np.arange(0.5,7.5, 1))

In [ ]:
# Do it with a biased coin..
b = 1.0 / 7.0
b1 = 2.0 / 7.0
results = pd.DataFrame(np.random.choice(outcomes, 1000, p=[b, b, b1, b, b, b]))
results.plot.hist(bins=np.arange(0.5,7.5, 1))

In [ ]:
# Do it for multiple events!
die1 = np.random.choice(outcomes, 100)
die2 = np.random.choice(outcomes, 100)
results = pd.DataFrame({'Die1': die1, 'Die2':die2})
results.head()

In [ ]:
# Need to add them up...
plt.figure(figsize = (12,5))
results['sum'] = results["Die1"] + results["Die2"]
results['sum'].plot.hist(bins=np.arange(1.5, 13.5, 1), density=True)

In [ ]:
# Default is with replacement but we can do without replacement..
people = ['Winona', 'Xanthippe', 'Yvonne', 'Zelda']
np.random.choice(people, 3, replace=False)

## Looking at Two Variables.

Let's roll two dice a bunch of times and see the resutls.


In [ ]:
die1 = np.random.choice(outcomes, 100)
die2 = np.random.choice(outcomes, 100)
results = pd.DataFrame({'Die1': die1, 'Die2':die2})

In [ ]:
counts = pd.crosstab(results['Die1'], results['Die2'])
counts

In [ ]:
joint = pd.crosstab(results['Die1'], results['Die2'], normalize=True)
joint

In [ ]:
# Now we can roll this up for either die to see it's distribution
joint.sum(axis=0)

In [ ]:
# Can also get marginals directly.
marginals = pd.crosstab(results['Die1'], results['Die2'], normalize=True, margins=True)
marginals

In [ ]:
# Finally, if we want conditional distributions we have to do a bit of work. Let's try to work out
# P(Die 1 is a 6 | Die 2 is a 5)

counts = pd.crosstab(results['Die1'], results['Die2'])
counts

In [ ]:
# We need to get the (Die 2 is a 5 row) and then look at the distribution there..

counts[5] / counts[5].sum()

## Using Simulation to Answer Probability Questions.

In CMPS 2170 we figured out closed form formulas for a set of mutually independent Bernoilli Trials.

* Bernoulli Trial: an experiment with two possible outcomes
* E.g., flip a coin results in two possible outcomes: head (𝐻) and tail (𝑇)
* Independent Bernoulli Trials: a sequence of Bernoulli trails that are mutually independent

* Example: What is the probability of the sequence HHHTT for a coin flip sequence with $p$ for H and $1-p$ for T? -- $p^3(1-p)^2$.

Recall: The probability of exactly $k$ successes in $n$ independent Bernoulli trials, with probability of success $p$ and probability of failure $q = 1 − p$, is $C(n,k)p^kq^{n-k}$ where $C(n,k)$ is $n$ choose $k$.

In [ ]:
# Setup a biased coin and flip it a bunch..
coin_results = np.random.choice(["Heads", "Tails"], 100, p=[0.75, 0.25])
coin_results

## A More complex Question..

* What is the probability of getting 60 or more heads if I flip 100 coins?
* Approximation through simulation:
  1. Figure out how to do one experiment (i.e., flip 100 coins).
  2. Run the experiment a bunch of times.
  3. Find the fraction of times where number of heads >= 60.

In [ ]:
# Flip 100 coins and count heads...
coin_results = np.random.choice(["Heads", "Tails"], 100, p=[0.75, 0.25])
print(coin_results == 'Heads')
print(np.count_nonzero(coin_results == 'Heads'))


In [ ]:
# Wrap it up and do it a bunch...
# Note we're using Numpy here for broadcasting -- numpy arrays are imuteable so 
# it's a tad more akaward in places..
n_reps = 10000

def exp():
    coin_results = np.random.choice(["Heads", "Tails"], 100, p=[0.75, 0.25])
    return np.count_nonzero(coin_results == 'Heads')

head_counts = np.array([])
for i in range(n_reps):
    head_counts = np.append(head_counts, exp())

In [ ]:
# Figure it out...
print(np.count_nonzero(head_counts >= 60))
print(np.count_nonzero(head_counts >= 60) / n_reps)

If we work out the math we need at least 60 H so we have to add up quite a few things...
$\sum^{100}_{k=60} C(100, k)p^kq^n-k$

In [ ]:
# Using simiulation we can also look at the trials.

head_counts = np.array([])
for i in range(n_reps):
    head_counts = np.append(head_counts, exp())

results = pd.DataFrame(head_counts)
results.plot.hist()

## Settle the Monty Hall Thing...

In [ ]:
def simulate_monty_hall():
    behind_picked_door = np.random.choice(['Car', 'Goat 1', 'Goat 2'])
    
    if behind_picked_door == 'Car':
        winning_strategy = 'Stay'
    else:
        winning_strategy = 'Switch'
        
    print(behind_picked_door, 'was behind the door. Winning strategy:', winning_strategy)
    return winning_strategy
simulate_monty_hall()

In [ ]:
# Run it a bunch...
n_repetitions = 10000

winning_strategies = np.array([])
for i in np.arange(n_repetitions):
    winning_strategy = simulate_monty_hall()
    winning_strategies = np.append(winning_strategies, winning_strategy)


In [ ]:
np.count_nonzero(winning_strategies == 'Switch') / n_repetitions

In [ ]:
np.count_nonzero(winning_strategies == 'Stay') / n_repetitions